In [13]:
# from PyPDF2 import PdfReader

# # Load the PDF file
# pdf_path = "2408.04948v1.pdf"
# reader = PdfReader(pdf_path)

# # Extract text from each page
# text = ""
# for page in reader.pages:
#     text += page.extract_text()

# # Print or save the extracted text
# print(text)


'HybridRAG: Integrating Knowledge Graphs and Vector Retrieval\nAugmented Generation for Efficient Information Extraction\nBhaskarjitSarmah BenikaHall RohanRao\nbhaskarjit.sarmah@blackrock.com bhall@nvidia.com rohrao@nvidia.com\nBlackRock,Inc. NVIDIA NVIDIA\nGurugram,India SantaClara,CA,USA SantaClara,CA,USA\nSunilPatel StefanoPasquali DhagashMehta\nsupatel@nvidia.com stefano.pasquali@blackrock.com dhagash.mehta@blackrock.com\nNVIDIA BlackRock,Inc. BlackRock,Inc.\nSantaClara,CA,USA NewYork,NY,USA NewYork,NY,USA\nABSTRACT AlthoughLLMshavesubstantialpotentialinfinancialapplica-\nExtractionandinterpretationofintricateinformationfromunstruc- tions,therearenotablechallengesinusingpre-trainedmodelsto\nturedtextdataarisinginfinancialapplications,suchasearnings extractinformationfromfinancialdocumentsoutsidetheirtraining\ncalltranscripts,presentsubstantialchallengestolargelanguage datawhilealsoreducinghallucination[7,8].Financialdocuments\nmodels (LLMs) even using the current best practices to 

In [1]:
from pdfminer.high_level import extract_text
text = extract_text('2408.04948v1.pdf')
with open("output.txt","w",encoding='utf-8') as file:
    file.write(text)

In [2]:
import re
# Apply the replacements
text_modified = re.sub(r'-\n', '', text)
text_modified = re.sub(r'(?<!\n)\n(?!\n)', ' ', text_modified)


In [30]:
text_modified

'4 2 0 2\n\ng u A 9\n\n] L C . s c [\n\n1 v 8 4 9 4 0 . 8 0 4 2 : v i X r a\n\nHybridRAG: Integrating Knowledge Graphs and Vector Retrieval Augmented Generation for Efficient Information Extraction\n\nBhaskarjit Sarmah bhaskarjit.sarmah@blackrock.com BlackRock, Inc. Gurugram, India\n\nBenika Hall bhall@nvidia.com NVIDIA Santa Clara, CA, USA\n\nRohan Rao rohrao@nvidia.com NVIDIA Santa Clara, CA, USA\n\nSunil Patel supatel@nvidia.com NVIDIA Santa Clara, CA, USA\n\nStefano Pasquali stefano.pasquali@blackrock.com BlackRock, Inc. New York, NY, USA\n\nDhagash Mehta dhagash.mehta@blackrock.com BlackRock, Inc. New York, NY, USA\n\nABSTRACT Extraction and interpretation of intricate information from unstructured text data arising in financial applications, such as earnings call transcripts, present substantial challenges to large language models (LLMs) even using the current best practices to use Retrieval Augmented Generation (RAG) (referred to as VectorRAG techniques which utilize vector data

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n\n", "\n\n",".","\n"],  # Prioritize splitting by paragraphs, then sentences, then spaces
    chunk_size = 500,  # Maximum size of each chunk
    chunk_overlap = 100  # Overlap to maintain context between chunks
)

In [4]:
chunks = text_splitter.split_text(text_modified)

In [86]:
chunks[3]

'. We introduce a novel approach based on a combination, called HybridRAG, of the Knowledge Graphs (KGs) based RAG techniques (called GraphRAG) and VectorRAG techniques to enhance question-answer (Q&A) systems for information extraction from financial documents that is shown to be capable of generating accurate and contextually relevant answers'

In [5]:
def clean_chunk(chunk):
    cleaned_chunk = chunk.strip() # remove leading/trailing whitespaces
    cleaned_chunk = re.sub(r'^\.+', '', cleaned_chunk) # remove leading fullstops
    cleaned_chunk = re.sub(r'\n+', ' ', cleaned_chunk) # remove \n\n
    cleaned_chunk = cleaned_chunk.strip()
    return cleaned_chunk


In [6]:
cleaned_chunk = [clean_chunk(chunk) for chunk in chunks]

In [7]:
cleaned_chunk

['4 2 0 2 g u A 9 ] L C . s c [ 1 v 8 4 9 4 0 . 8 0 4 2 : v i X r a HybridRAG: Integrating Knowledge Graphs and Vector Retrieval Augmented Generation for Efficient Information Extraction Bhaskarjit Sarmah bhaskarjit.sarmah@blackrock.com BlackRock, Inc. Gurugram, India Benika Hall bhall@nvidia.com NVIDIA Santa Clara, CA, USA Rohan Rao rohrao@nvidia.com NVIDIA Santa Clara, CA, USA Sunil Patel supatel@nvidia.com NVIDIA Santa Clara, CA, USA',
 'Sunil Patel supatel@nvidia.com NVIDIA Santa Clara, CA, USA Stefano Pasquali stefano.pasquali@blackrock.com BlackRock, Inc. New York, NY, USA Dhagash Mehta dhagash.mehta@blackrock.com BlackRock, Inc. New York, NY, USA',
 'ABSTRACT Extraction and interpretation of intricate information from unstructured text data arising in financial applications, such as earnings call transcripts, present substantial challenges to large language models (LLMs) even using the current best practices to use Retrieval Augmented Generation (RAG) (referred to as VectorRAG t

In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('BAAI/bge-small-en-v1.5')

d:\Internship\Chat_With_Docs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
chunk_embeddings = model.encode(cleaned_chunk)

In [91]:
chunk_embeddings.shape

(145, 384)

In [10]:
import pandas as pd

assert len(chunk_embeddings) == len(cleaned_chunk)

df = pd.DataFrame(
    {
        "Text" : cleaned_chunk,
        "Embeddings" : [row.tolist() for row in chunk_embeddings]
    }
)

df.head()


,Text,Embeddings
0,4 2 0 2 g u A 9 ] L C . s c [ 1 v 8 4 9 4 0 . ...,"[-0.019694823771715164, 0.03228660300374031, -..."
1,Sunil Patel supatel@nvidia.com NVIDIA Santa Cl...,"[0.001957190688699484, -0.0019113713642582297,..."
2,ABSTRACT Extraction and interpretation of intr...,"[-0.03259172663092613, 0.024141255766153336, -..."
3,We introduce a novel approach based on a combi...,"[-0.0541795939207077, 0.0531788133084774, -0.0..."
4,Using experiments on a set of financial earnin...,"[-0.03212740644812584, 0.04043416306376457, -0..."


In [100]:
query = "What is hybridRAG?"

embedded_query = model.encode(clean_chunk(query))


In [14]:
import numpy as np
def cosine_similarity(a,b):
    a = np.array(a)
    b = np.array(b)

    dot_product = np.dot(a,b)

    mag_a = np.linalg.norm(a)
    mag_b = np.linalg.norm(b)

    similarity = dot_product / (mag_a * mag_b)
    return similarity

In [101]:
print(cosine_similarity(embedded_query,df["Embeddings"][0]))

0.5949147382728737


In [102]:
df["similarity"] = df["Embeddings"].apply(lambda x : cosine_similarity(x, embedded_query))


In [103]:
df.head()

,Text,Embeddings,similarity
0,4 2 0 2 g u A 9 ] L C . s c [ 1 v 8 4 9 4 0 . ...,"[-0.019694823771715164, 0.03228660300374031, -...",0.594915
1,Sunil Patel supatel@nvidia.com NVIDIA Santa Cl...,"[0.001957190688699484, -0.0019113713642582297,...",0.462932
2,ABSTRACT Extraction and interpretation of intr...,"[-0.03259172663092613, 0.024141255766153336, -...",0.541232
3,We introduce a novel approach based on a combi...,"[-0.0541795939207077, 0.0531788133084774, -0.0...",0.603789
4,Using experiments on a set of financial earnin...,"[-0.03212740644812584, 0.04043416306376457, -0...",0.606146


In [104]:
results = df.sort_values("similarity", ascending= False)

In [105]:
results.head()

,Text,Embeddings,similarity
21,"In the present work, we propose a combination ...","[-0.04629891738295555, -0.014486481435596943, ...",0.756713
110,The results of our comparative analysis reveal...,"[-0.004982789978384972, -0.03970677778124809, ...",0.756547
115,Overall GraphRAG performs better in extractive...,"[-0.025407111272215843, -0.03301198408007622, ...",0.751247
114,"Despite this trade-off, HybridRAG’s superior p...","[-0.025110999122262, 0.030811332166194916, -0....",0.740144
112,"Overall, these results suggest that GraphRAG o...","[-0.02828267589211464, -0.038765255361795425, ...",0.721592


In [107]:
results["Text"][21]

'In the present work, we propose a combination of VectorRAG and GraphRAG, called HybridRAG, to retrieve the relevant information from external documents for a given query to the LLM that brings advantages of both the RAGs together to provide demonstrably more accurate answers to the queries.'

In [112]:
top_k = results["Text"].head(3).tolist()

In [11]:
import cohere

co = cohere.Client("3Ey34YJTJS112veQg08hrjU8vmch67EhBWeiWBNU")



In [48]:
query = "What is the summary of this paper?"
embedded_query = model.encode(clean_chunk(query))
df["similarity"] = df["Embeddings"].apply(lambda x : cosine_similarity(x, embedded_query))
results = df.sort_values("similarity", ascending= False)
top_k = results["Text"].head(3).tolist()

In [43]:
results.head()

,Text,Embeddings,similarity
14,Due to the complexities in analyzing financial...,"[-0.007800274528563023, 0.021058620885014534, ...",0.651927
58,"""Consider the given context and following stat...","[-0.061397381126880646, 0.048492200672626495, ...",0.649732
119,"Furthermore, the quality of the retrieved cont...","[0.01591600477695465, 0.0022000079043209553, -...",0.646635
76,Table 1: Summary Statistics for the call trans...,"[-0.0033405821304768324, -0.00464419461786747,...",0.643603
40,A schematic diagram of the retrieval methodolo...,"[0.021049920469522476, -0.019680125638842583, ...",0.634759


In [51]:
# Build the formatted context
formatted_input = "Context:\n"
for idx, result in enumerate(top_k, 1):
    formatted_input += f"{idx}. {result}\n"

full_prompt = f"""
You are given a context based on multiple sources. Your task is to answer the following question based on the provided context. 
If the context does not contain relevant information or does not match the query, please respond with "I cannot answer based on the given context."

{formatted_input}

Question:
{query}

Instructions:
- Use your knowledge to check which context is most relevant to the question being asked.
- Provide a concise, clear, and relevant answer based on the context.
- Keep your answer short and dont ask additional questions.
"""


In [52]:
print(full_prompt)


You are given a context based on multiple sources. Your task is to answer the following question based on the provided context. 
If the context does not contain relevant information or does not match the query, please respond with "I cannot answer based on the given context."

Context:
1. Table 1: Summary Statistics for the call transcript documents used in the present work. These call transcripts documents consist of questions and answers between financial analysts and the company representatives for the respective companies, hence, there already exist certain Q&A pairs within these documents along with additional text. We examined the earnings reports within the Nifty50 universe, systematically curated a comprehensive array of randomly selected 400
2. Table 1 summarizes basic statistics of the documents we will be experimenting with in the remainder of this work. Number of companies/documents Average number of pages Average number of questions Average number of tokens 50 27 16 60,00

In [53]:

response = co.generate(
    model="command",
    prompt=full_prompt,
    max_tokens=200
)

In [54]:
response.generations[0].text

' This paper seems to be examining the need for a more complex analysis of LLM-generated financial contexts, amidst a call for more sophisticated methods that can effectively integrate and process the detailed and domain-specific information found in financial documents, ensuring more reliable and accurate results for informed decision-making in the future. '